In [101]:
import json
import fastai
f = open("D:\\Tasks\\BIRD_CLEF\\birdclef-2022\\scored_birds.json")
T = json.load(f)
L=list(T)
print(L)
print(len(L))
print(type(L))

['akiapo', 'aniani', 'apapan', 'barpet', 'crehon', 'elepai', 'ercfra', 'hawama', 'hawcre', 'hawgoo', 'hawhaw', 'hawpet1', 'houfin', 'iiwi', 'jabwar', 'maupar', 'omao', 'puaioh', 'skylar', 'warwhe1', 'yefcan']
21
<class 'list'>


In [103]:
import os
  
# Get the list of all files and directories
# in the root directory
path = "/"
dir_list = os.listdir("birdclef-2022\\train_audio")
bird_map = []
for idx,dir1 in enumerate(dir_list):
    if(dir1 in L):
        bird_map.append(idx)
print(bird_map)
print(len(bird_map))

[3, 6, 7, 9, 44, 46, 47, 60, 62, 63, 64, 65, 67, 70, 72, 90, 101, 111, 131, 141, 150]
21


In [3]:
from fastai.vision.all import *
import torchaudio
from joblib import Parallel, delayed
from tqdm.notebook import tqdm
from torchvision.utils import save_image
base_folder = Path('birdclef-2022')
train = pd.read_csv('birdclef-2022/train_metadata.csv')
items = get_files(base_folder, extensions='.ogg')

In [152]:
#Custom DataLoader
from torch.utils.data import Dataset
class BirdClefDataset(Dataset):

    def __init__(self, annotations_file, audio_dir,transformation):
        self.train_meta = pd.read_csv(annotations_file)
        self.audio_dir = audio_dir
        self.target_sample_rate = 32000
        self.device = "cuda"
        self.transformation = transformation.to(self.device)
        self.num_samples=1800
        self.num_classes = self._create_onehot()

    def _create_label(self,bird):
        lbl = [0]*self.num_classes
        lbl[self.enc[bird]] = 1
        return lbl[:]
    
    def depredict(self,label,bird):
        return label[self.enc[bird]]
    
    def _create_onehot(self):
        classes = set()
        self.enc = {} ##
        for i in self.train_meta.primary_label:
            classes.add(i)
        for idx,clss in enumerate(classes):
            self.enc[clss] = idx
        return len(classes)
    
    def __len__(self):
        return len(self.train_meta)

    def __getitem__(self, index):
        audio_sample_path = self._get_audio_sample_path(index)
        label = self._get_audio_sample_label(index)
        signal, sr = torchaudio.load(audio_sample_path)
        signal = signal.to(self.device)
        #signal -> (num_channels,samples) -> (1,sr)
#         signal = self._resample_if_necessary(signal,sr)
        signal = self._mix_down_if_necessary(signal)
        signal = self._rightpad_if_necessary(signal)
        signal = self._cut_if_necessary(signal)
        signal = self.transformation(signal)
        return signal, torch.from_numpy(np.array(self._create_label(label),dtype=np.float32)).to(device)
    
    def _resample_if_necessary(self,signal,sr):
        if sr!= self.target_sample_rate:
            resampler = torchaudio.transforms.Resample(sr,self.target_sample_rate)
            signal = resample(signal)
        return signal
    def _mix_down_if_necessary(self,signal): # turn multiple channels into mono, do we have multiple channels?
        if(signal.shape[0]>1):
            signal = torch.mean(input = signal, dim=0,keepdim=True)
        return signal
    def _cut_if_necessary(self,signal):
        # signal(tensor)[num_channels=1,number of samples]
        if(signal.shape[1]>self.num_samples):
            signal = signal[:,:self.num_samples]
        return signal
    def _rightpad_if_necessary(self,signal):
        length_signal = signal.shape[1]
        if(length_signal < self.num_samples):
            num_missing_samples = self.num_samples - length_signal
            last_dim_padding = (0,num_missing_samples)
            signal = torch.nn.functional.pad(signal,last_dim_padding)
        return signal
        

    def _get_audio_sample_path(self, index):
        fold = f"fold{self.train_meta.iloc[index, 0]}"
        bird,file = self.train_meta.iloc[index, 12].split("/")
        path = os.path.join(self.audio_dir,bird,file)
        return path

    def _get_audio_sample_label(self, index):
        return self.train_meta.iloc[index, 0]

In [168]:
import torch
import numpy as np
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
random_seed =  9
SAMPLE_RATE = 32000
mel_spectrogram = torchaudio.transforms.MelSpectrogram(sample_rate=SAMPLE_RATE
                                                      ,n_fft=1024,
                                                      hop_length=512,
                                                      n_mels= 64)
dataset = BirdClefDataset("D:\\Tasks\\BIRD_CLEF\\birdclef-2022\\train_metadata.csv",
                            "birdclef-2022\\train_audio", mel_spectrogram)

# Creating data indices for training and validation splits:
validation_split = 0.2
shuffle_dataset = True
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=200,)



In [141]:
train_meta = pd.read_csv("D:\\Tasks\\BIRD_CLEF\\birdclef-2022\\train_metadata.csv")
print(len(train_meta.primary_label.unique()))
print(dataset._create_label("comgal1"))

152
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [154]:
for imgs, labels in train_loader:
    print("Batch of images has shape: ",imgs.size())
    print("Batch of labels has shape: ", labels.size())
    break

Batch of images has shape:  torch.Size([40, 1, 64, 4])
Batch of labels has shape:  torch.Size([40, 152])


for imgs, labels in data_loader:
    print("Batch of images has shape: ",imgs.shape)
    print("Batch of labels has shape: ", labels.shape)

In [155]:
class BirdClefModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.dense_layers = nn.Sequential(
            nn.Linear(256,256) # simple dense layer
            , nn.ReLU(),
            nn.Linear(256,152)
        )
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self,input_data):
        flattened_data = self.flatten(input_data)
        logits = self.dense_layers(flattened_data)
        predictions = self.softmax(logits)
        return predictions

In [169]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=5, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(12)
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=5, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(12)
        self.pool = nn.MaxPool2d(2,2)
        self.conv4 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=5, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(24)
        self.conv5 = nn.Conv2d(in_channels=24, out_channels=24, kernel_size=5, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(24)
        self.fc1 = nn.Linear(24*10*10, 152)

    def forward(self, input):
        output = F.relu(self.bn1(self.conv1(input)))      
        output = F.relu(self.bn2(self.conv2(output)))     
        output = self.pool(output)                        
        output = F.relu(self.bn4(self.conv4(output)))     
        output = F.relu(self.bn5(self.conv5(output)))     
        output = output.view(-1, 24*10*10)
        output = self.fc1(output)


In [ ]:
import torch.optim as optim
device ='cuda'
def train_one_epoch(model,data_loader,loss_fn,optimiser,device):
    device ='cuda'
    for inputs,targets in tqdm(data_loader):
        inputs,targets = inputs.to(device),targets.to(device)
        #Loss calculations
        predictions = model(inputs)
        #calculate loss and backpropagate , using gd for updating 
        loss = loss_fn(predictions,targets)
    
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
    print(f"Loss: {loss.item()}")

     
def train(model,data_loader,loss_fn,optimiser,device,epochs):
    for i in range(epochs):
        print(f"Epoch {i}:")
        train_one_epoch(model,data_loader,loss_fn,optimiser,device)
        print("------------")
    print("Training is done")


EPOCHS = 5
LEARNING_RATE = 0.001
#loss 
loss_fn = nn.functional.cross_entropy
model = Network().to('cuda')
import time
print(time.time())
start = time.time()
optimiser = optim.Adam(model.parameters(),lr=LEARNING_RATE)
train(model,train_loader,loss_fn,optimiser,device,EPOCHS)
print(time.time()-start)

1653405413.6072958
Epoch 0:


  0%|          | 0/75 [00:00<?, ?it/s]

In [159]:
torch.save(model.state_dict(),"LRUmodel_1.pth")
model_val = BirdClefModel()
trained_state_dict = torch.load("LRUmodel_1.pth")
model_val.load_state_dict(trained_state_dict)
# validation_loader = torch.utils.data.DataLoader(dataset, batch_size=2,
#                                                 sampler=valid_sampler)
# valid_loss = 0.0
# model.eval()     # Optional when not using Model Specific layer
# for data, labels in validation_loader:
#     if torch.cuda.is_available():
#         data, labels = data.cuda(), labels.cuda()

#     target = model(data)
#     print(target.shape)
#     print(labels.shape)
#     loss = loss_fn(target,labels)
#     valid_loss = loss.item() * data.size(0)
#     print(loss)

# print(f'Epoch {e+1} \t\t Training Loss: {train_loss / len(trainloader)} \t\t Validation Loss: {valid_loss / len(validloader)}')

<All keys matched successfully>

In [160]:
#Here we split the test audio file
test_audio_dir = 'birdclef-2022/test_soundscapes/'
file_list = [f.split('.')[0] for f in sorted(os.listdir(test_audio_dir))]
with open('birdclef-2022/scored_birds.json') as sbfile:
    scored_birds = json.load(sbfile)
pred = {'row_id': [], 'target': []}
from pydub import AudioSegment
from pydub.utils import make_chunks 
sound = AudioSegment.from_file("birdclef-2022/test_soundscapes/soundscape_453028782.ogg")

chunk_length_ms = 5000 # pydub calculates in millisec 
chunks = make_chunks(sound,chunk_length_ms)
for i, chunk in enumerate(chunks): 
    chunk_name = "{0}.ogg".format(i) 
    print ("exporting", chunk_name) 
    chunk.export(chunk_name, format="ogg") 

exporting 0.ogg
exporting 1.ogg
exporting 2.ogg
exporting 3.ogg
exporting 4.ogg
exporting 5.ogg
exporting 6.ogg
exporting 7.ogg
exporting 8.ogg
exporting 9.ogg
exporting 10.ogg
exporting 11.ogg


In [89]:
import librosa
import numpy as np

def _resample_if_necessary(signal,sr):
        if sr!= 32000:
            resampler = torchaudio.transforms.Resample(sr,32000)
            signal = resample(signal)
        return signal
def _mix_down_if_necessary(signal): # turn multiple channels into mono, do we have multiple channels?
    if(signal.shape[0]>1):
        signal = torch.mean(input = signal, dim=0,keepdim=True)
    return signal
def _cut_if_necessary(signal):
    # signal(tensor)[num_channels=1,number of samples]
    if(signal.shape[1]>1800):
        signal = signal[:,:1800]
    return signal

def features_extractor(file):
    transformation = mel_spectrogram.to('cuda')
    signal, sr = torchaudio.load(file)
    signal = signal.to('cuda')
    #signal -> (num_channels,samples) -> (1,sr)
    signal = _resample_if_necessary(signal,sr)
    signal = _mix_down_if_necessary(signal)
    signal = _cut_if_necessary(signal)
    signal = transformation(signal)
    return signal


import os
extracted_features_=[]
for i in range(0,12):
    file_name = os.path.join(os.path.abspath("./"),str(i) + ".ogg")
    data=features_extractor(file_name)
    extracted_features_.append([data])

In [90]:
import pandas as pd
extracted_features_df_=pd.DataFrame(extracted_features_,columns=['feature'])
X_pro = extracted_features_df_['feature']

In [91]:
with open('birdclef-2022/scored_birds.json') as sbfile:
    scored_birds = json.load(sbfile)

In [92]:
model.eval()
predict__=model( torch.cat(X_pro.tolist()))

In [120]:

# This is where we will store our results
pred = {'row_id': [], 'target': []}

# Process audio files and make predictions
bird_map = [3, 6, 7, 9, 44, 46, 47, 60, 62, 63, 64, 65, 67, 70, 72, 90, 101, 111, 131, 141, 150]
for afile in file_list:
    
    # Complete file path
    path = test_audio_dir + afile + '.ogg'
    
    # Open file with librosa and split signal into 5-second chunks
    # sig, rate = librosa.load(path)
    # ...
    
    # Let's assume we have a list of 12 audio chunks (1min / 5s == 12 segments)
    chunks = [[] for i in range(12)]
    
    # Make prediction for each chunk
    # Each scored bird gets a random value in our case
    # since we don't actually have a model
    for i in range(len(chunks)):        
        chunk_end_time = (i + 1) * 5
        j=0
        for bird in scored_birds:
            
            # This is our random prediction score for this bird
            score = predict__[i][j]
            j=j+1
            # Assemble the row_id which we need to do for each scored bird
            row_id = afile + '_' + bird + '_' + str(chunk_end_time)
            # Put the result into our prediction dict and
            # apply a "confidence" threshold of 0.5
            pred['row_id'].append(row_id)
            pred['target'].append(True if score > 0.00006 else False)

In [121]:
import torch.optim as optim
device ='cuda'
def train_one_epoch(model,data_loader,loss_fn,optimiser,device):
    device ='cuda'
    for inputs,targets in tqdm(data_loader):
        inputs,targets = inputs.to(device),targets.to(device)
        #Loss calculations
        while(inputs.shape[2]<5):
            inputs = torch.cat([inputs,[0]*64])
        predictions = model(inputs)
        #calculate loss and backpropagate , using gd for updating 
        loss = loss_fn(predictions,targets)
    
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
    print(f"Loss: {loss.item()}")

     
def train(model,data_loader,loss_fn,optimiser,device,epochs):
    for i in range(epochs):
        print(f"Epoch {i}:")
        train_one_epoch(model,data_loader,loss_fn,optimiser,device)
        print("------------")
    print("Training is done")


EPOCHS = 10
LEARNING_RATE = 0.001
#loss 
loss_fn = nn.functional.cross_entropy
model = BirdClefModel().to('cuda')
optimiser = optim.Adam(model.parameters(),lr=LEARNING_RATE)
train(model,train_loader,loss_fn,optimiser,device,EPOCHS)

Epoch 0:


  0%|          | 0/298 [00:00<?, ?it/s]

Loss: 5.026534557342529
------------
Epoch 1:


  0%|          | 0/298 [00:00<?, ?it/s]

Loss: 5.033028602600098
------------
Epoch 2:


...

Loss: 4.944440841674805
------------
Epoch 8:


  0%|          | 0/298 [00:00<?, ?it/s]

Loss: 5.032506942749023
------------
Epoch 9:


  0%|          | 0/298 [00:00<?, ?it/s]

Loss: 5.027227878570557
------------
Training is done


In [116]:
results = pd.DataFrame(pred, columns = ['row_id', 'target'])

# Quick sanity check
print(results.head()) 
    
# Convert our results to csv
results.to_csv("submission.csv", index=False)

                          row_id  target
0  soundscape_453028782_akiapo_5   False
1  soundscape_453028782_aniani_5   False
2  soundscape_453028782_apapan_5   False
3  soundscape_453028782_barpet_5   False
4  soundscape_453028782_crehon_5   False
